# Baseline for Ubiquant Market Prediction using ANN

**Thanks to notebook contributors for make baseline for my notebook**
- https://www.kaggle.com/code/sohier/competition-api-detailed-introduction/notebook
- https://www.kaggle.com/code/pythonash/end-to-end-simple-and-powerful-dnn-with-leakyrelu
- https://www.kaggle.com/code/robikscube/fast-data-loading-and-low-mem-with-parquet-files

First import libraries and dataset.

In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow.keras as keras
import tensorflow as tf
import os
import si

In [11]:
df = pd.read_parquet('D:/kaggle_datasets/ubiquant-parquet/train_low_mem.parquet')

# Preprocess dataset

Because of it's baseline I use only f_n for features for X_train.

In [12]:
index_col = df.drop(["row_id", "time_id", "investment_id", "target"], axis=1).columns
X_train = df[index_col]
X_train = X_train.astype("float16")

In [13]:
y_train = df["target"]

# Making model

I make simple model using Dense layer and BatchNormalizaion without hyper parameter tuning.

In [16]:
Input = keras.layers.Input(shape=X_train.shape[1])
x = keras.layers.Dense(128, activation="relu")(Input)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(64, activation="relu")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(32, activation="relu")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(16, activation="relu")(x)
x = keras.layers.BatchNormalization()(x)
Output = keras.layers.Dense(1, activation="linear")(x)

model = keras.models.Model(inputs = Input, outputs = Output)

In [18]:
model.compile(loss="mse", optimizer="adam", metrics=["mse"])
model.fit(X_train, y_train, validation_split=0.2, epochs=5)f

Train on 2513128 samples, validate on 628282 samples
Epoch 1/5
2513128/2513128 [==============================] - 357s 142us/sample - loss: 0.8404 - mse: 0.8404 - val_loss: 0.8095 - val_mse: 0.8095
Epoch 2/5
2513128/2513128 [==============================] - 364s 145us/sample - loss: 0.8333 - mse: 0.8333 - val_loss: 0.8134 - val_mse: 0.8134
Epoch 3/5
2513128/2513128 [==============================] - 374s 149us/sample - loss: 0.8303 - mse: 0.8303 - val_loss: 0.8103 - val_mse: 0.8103
Epoch 4/5
2513128/2513128 [==============================] - 445s 177us/sample - loss: 0.8283 - mse: 0.8283 - val_loss: 0.8175 - val_mse: 0.8175
Epoch 5/5
2513128/2513128 [==============================] - 418s 166us/sample - loss: 0.8264 - mse: 0.8264 - val_loss: 0.8800 - val_mse: 0.8800


# 특성 공학 및 하이퍼 파라미터 서칭 

In [23]:
Input = keras.layers.Input(shape=X_train.shape[1])
x = keras.layers.Dense(128)(Input)
x = keras.layers.LeakyReLU(alpha=0.2)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(64)(x)
x = keras.layers.LeakyReLU(alpha=0.2)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(32)(x)
x = keras.layers.LeakyReLU(alpha=0.2)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(16)(x)
x = keras.layers.LeakyReLU(alpha=0.2, )(x)
x = keras.layers.BatchNormalization()(x)
Output = keras.layers.Dense(1, activation="linear")(x)

model_leak = keras.models.Model(inputs = Input, outputs = Output)

In [24]:
model_leak.compile(loss="mse", optimizer="adam", metrics=["mse"])
history = model_leak.fit(X_train, y_train, validation_split=0.2, epochs=5)

Train on 2513128 samples, validate on 628282 samples
Epoch 1/5
2513128/2513128 [==============================] - 469s 187us/sample - loss: 0.8408 - mse: 0.8408 - val_loss: 0.8120 - val_mse: 0.8120
Epoch 2/5
2513128/2513128 [==============================] - 421s 167us/sample - loss: 0.8347 - mse: 0.8347 - val_loss: 0.8096 - val_mse: 0.8096
Epoch 3/5
2513128/2513128 [==============================] - 427s 170us/sample - loss: 0.8323 - mse: 0.8323 - val_loss: 0.8087 - val_mse: 0.8087
Epoch 4/5
2513128/2513128 [==============================] - 347s 138us/sample - loss: 0.8308 - mse: 0.8308 - val_loss: 0.8127 - val_mse: 0.8127
Epoch 5/5
2513128/2513128 [==============================] - 347s 138us/sample - loss: 0.8296 - mse: 0.8296 - val_loss: 0.8107 - val_mse: 0.8107


In [7]:
model.save("model.h5")

# Submission

In [8]:
import ubiquant
model = tf.keras.models.load_model('model.h5')
env = ubiquant.make_env()   
iter_test = env.iter_test()    
for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df["target"] = model.predict(test_df[index_col].astype("float16"))
    env.predict(sample_prediction_df)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
